In [1]:
%%capture
%run ./core/Track_Processor_Methods.ipynb
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings; warnings.simplefilter('ignore')
import math
from matplotlib.patches import Circle
import itertools
import numpy.linalg
from IPython.display import clear_output, display
from matplotlib import colors
matplotlib.rc('font',family='Arial')
import os
# import ipywidgets as widgets
# from ipywidgets import interact, interact_manual, interactive, Dropdown
# from IPython.display import HTML, display


ppmm = 2.75 #Pixels per millimeter
min_frames = 2 #Minimum frames to be considered for the track
fps = 30 # Video Framerate
flw_thres = 20 #Additional flower radius
data_points = 9 # Number of data collection point
min_duration_on_flower = 5 # Min duration (frames) on a flower to be considered as a visit

In [70]:
track_input_directory = '/Users/mrat0010/Documents/GitHub/Polytrack_WIP/data/tracks/processed_tracks_cvpr/'
flower_input_directory = '/Users/mrat0010/Documents/GitHub/Polytrack_WIP/data/tracks/flowers_cvpr/'

summary_data_directory = './data/tracks/summary_data/'
plot_directory = './Graphs/'

csv_tracks = get_video_list(track_input_directory, 'csv')
print('Number of tracks: ', len(csv_tracks))

position_list = [115, 117, 208, 312, 317, 409, 417, 710,810]

Number of tracks:  43


In [11]:
insect_tracks = pd.DataFrame()
track_list =  []
frameN = []
num_bees = 0
for track_ in csv_tracks:
    t  = generateTrackNumber(track_)
    csv_name = str(track_)+'.csv'

    try:
        track_number = int(str(t))
        track1 = pd.read_csv(track_input_directory+str(csv_name))
        # Get the length of the dataframe
        len_track = len(track1)
        if len_track > 5:
                
            frameN = np.append(frameN,len(track1)) 
            track_list = np.append(track_list,int(track_number))

            track1= track1.rename(columns={'Unnamed: 0': 'frame_'+str(t)})
            track1 = track1.set_index('frame_'+str(t))
            track1 = track1.rename(columns={'nframe': 'nframe_'+str(t)})
            # track1 = track1.rename(columns={'absframe': 'absframe_'+str(t)})
            track1 = track1.rename(columns={'x0': 'x0_'+str(t)})
            track1 = track1.rename(columns={'y0': 'y0_'+str(t)})
            track1 = track1.rename(columns={'area': 'area_'+str(t)})
            track1 = track1.rename(columns={'y_adj': 'yadj_'+str(t)})
            track1 = track1.rename(columns={'flower': 'flower_'+str(t)})
            track1 = track1.rename(columns={'visit_num': 'visit_num_'+str(t)})
            # track1 = track1.rename(columns={'sx': 'sx_'+str(t)})
            # track1 = track1.rename(columns={'sy': 'sy_'+str(t)})
            # track1 = track1.rename(columns={'speeds': 'speeds_'+str(t)})
            window_size = 3  # adjust the window size as needed
            track1['sx_'+str(t)] = track1['x0_'+str(t)].rolling(window=window_size, min_periods=1).mean()
            track1['sy_'+str(t)] = track1['y0_'+str(t)].rolling(window=window_size, min_periods=1).mean()    



            insect_tracks = pd.concat([insect_tracks, track1['nframe_'+str(t)], track1['x0_'+str(t)], track1['y0_'+str(t)], track1['area_'+str(t)],
                                    track1['yadj_'+str(t)],track1['flower_'+str(t)],track1['visit_num_'+str(t)],track1['sx_'+str(t)],track1['sy_'+str(t)]],axis=1) 
            
        else:
            pass
    
    except:
        print('Error in track: ', track_)

insect_tracks.head() 

#Group tracks based on the data collection position

tracks_and_dcp = [[] for i in range(data_points)]
for t in track_list: tracks_and_dcp[getDataPoint(str(int(t)))-1].append(t)

    
# Save the combined track file to csv

insect_tracks.to_csv(summary_data_directory + 'all_tracks.csv')
track_list_write = open(summary_data_directory+"track_list.txt", "w")
for track in track_list:
    track_list_write.write(str(int(track)) + "\n")
track_list_write.close()


In [12]:
def generateFlowerPosition0(csv_name):
    fp = csv_name.split('_')[2]
  
    return int(fp)

In [73]:
flower_input_directory0 = '/Users/mrat0010/Documents/GitHub/Polytrack_WIP/data/tracks/flowers_cvpr/'
flowers_csv = get_video_list('/Users/mrat0010/Documents/GitHub/Polytrack_WIP/data/tracks/flowers_cvpr/', 'csv')
print(flowers_csv)
flower_positions = pd.DataFrame()
# position_list = []
for f in flowers_csv:
    csv_name = str(f)+'.csv'
    fpc = generateFlowerPosition(f)

    flower1 = pd.read_csv(flower_input_directory0+str(csv_name))

    # get the unique flower_num values
    flower_nums = flower1['flower_num'].unique()

    # Get the max value of index for each flower_num
    max_nframe = flower1.groupby(['flower_num'])['Unnamed: 0'].max()


    # create a new dataframe by extracting all the columns from the original dataframe for the max_nframe index
    flower2 = flower1.loc[flower1.groupby('flower_num')['Unnamed: 0'].idxmax()]

    flower2 = flower2.rename(columns={'flower_num': 'flower_num_'+str(fpc)})
    flower2 = flower2.rename(columns={'x0': 'x0_'+str(fpc)})
    flower2 = flower2.rename(columns={'y0': 'y0_'+str(fpc)})
    # flower2 = flower2.rename(columns={'y_adj': 'y_adj_'+str(fpc)})
    flower2 = flower2.rename(columns={'radius': 'radius_'+str(fpc)})
    # flower2 = flower2.rename(columns={'Total_time': 'Total_time_'+str(fpc)})
    # flower2 = flower2.rename(columns={'Total_visits': 'Total_visits_'+str(fpc)})

    flower_positions = pd.concat([flower_positions, flower2['flower_num_'+str(fpc)], flower2['x0_'+str(fpc)], flower2['y0_'+str(fpc)]
                                     , flower2['radius_'+str(fpc)]],  axis=1)


flower_positions

['flowers_115', 'flowers_117', 'flowers_208', 'flowers_312', 'flowers_317', 'flowers_409', 'flowers_417', 'flowers_710', 'flowers_810']


flower_num_115  x0_115  y0_115  radius_115  flower_num_117  x0_117  y0_117  \
0             0.0   571.0   406.0        39.0               0     855     760   
1             1.0   608.0   697.0        19.0               1     964     819   
2             2.0   748.0   869.0        39.0               2    1186     799   
3             3.0  1127.0   124.0        27.0               3    1261     888   
4             4.0  1206.0   201.0        32.0               4    1407     660   
5             NaN     NaN     NaN         NaN               5    1601     923   
6             NaN     NaN     NaN         NaN               6    1830     476   
7             NaN     NaN     NaN         NaN               7     155     224   

   radius_117  flower_num_208  x0_208  ...  y0_417  radius_417  \
0          23             0.0   584.0  ...   613.0        24.0   
1          20             1.0   784.0  ...   986.0        26.0   
2          34             2.0   886.0  ...   946.0        34.0   
3          31             3.0   900.0  ...   621.0        30.0   
4          29             4.0   210.0  ...   754.0        32.0   
5          28             NaN     NaN  ...   720.0        34.0   
6          19             NaN     NaN  ...   643.0        30.0   
7          20             NaN     NaN  ...     NaN         NaN   

   flower_num_710  x0_710  y0_710  radius_710  flower_num_810  x0_810  y0_810  \
0             0.0   620.0   808.0        39.0             0.0  1258.0   961.0   
1             1.0  1224.0   525.0        35.0             1.0   341.0   675.0   
2             2.0  1513.0   131.0        37.0             2.0  1432.0   825.0   
3             3.0   734.0   710.0        25.0             3.0  1406.0   695.0   
4             NaN     NaN     NaN         NaN             NaN     NaN     NaN   
5             NaN     NaN     NaN         NaN             NaN     NaN     NaN   
6             NaN     NaN     NaN         NaN             NaN     NaN     NaN   
7             NaN     NaN     NaN         NaN             NaN     NaN     NaN   

   radius_810  
0        26.0  
1        38.0  
2        45.0  
3        31.0  
4         NaN  
5         NaN  
6         NaN  
7         NaN  

[8 rows x 36 columns]

In [78]:
from matplotlib.patches import Circle
from matplotlib.lines import Line2D


#set figure size
# plt.style.use('default')
plt.figure(figsize=(19,10))
n_tracks = 0
for t in track_list:
    # if (getRecordDate(str(t)) in dates_to_plot) and (getSpeciesNumber(t) in species_to_plot):
    plt.plot((insect_tracks['sx_%d'%(int(t))]/ppmm), ((insect_tracks['sy_%d'%(int(t))]/ppmm)), color =getPlotColor(int(t)))
    plt.scatter((insect_tracks['sx_%d'%(int(t))]/ppmm), ((insect_tracks['sy_%d'%(int(t))]/ppmm)), color =getPlotColor(int(t)), s=1)

plt.xlim(0,1920/ppmm)
plt.ylim(0,1080/ppmm)
    #         frame_counts.append(len(insect_tracks['sx_%d'%(int(t))].dropna().tolist()))
    #         n_tracks += 1
    #     else:
    #         pass

flower_count = 0
ax = plt.gca()
for d in position_list:
    for row in flower_positions['x0_'+str(d)].dropna().index.tolist():
        flower_count += 1
        ax.add_patch(Circle((flower_positions['x0_'+str(d)][row]/ppmm, 1080/ppmm -flower_positions['y0_'+str(d)][row]/ppmm), radius = (flower_positions['radius_'+str(d)][row]+flw_thres)/ppmm, color = 'tab:olive', fill=True, alpha = 0.2))
        ax.add_patch(Circle((flower_positions['x0_'+str(d)][row]/ppmm, 1080/ppmm -flower_positions['y0_'+str(d)][row]/ppmm), radius = (flower_positions['radius_'+str(d)][row]+flw_thres)/ppmm, color = 'k', fill=False, alpha = 0.8))

# Add a legend in the bottom center with custom labels for insect species
legend_elements = [Line2D([0], [0], linestyle='-', color=getPlotColor(11513300100), label='Honeybee (T = '+str(len(honeybee_tracks))+")", markerfacecolor='k', markersize=10),
                   Line2D([0], [0], linestyle='-', color=getPlotColor(11513300101), label='Syrphidae (T = '+str(len(hoverfly_tracks))+")",markerfacecolor='k', markersize=10),
                   Line2D([0], [0], linestyle='-', color=getPlotColor(11513300102), label='Lepidoptera (T = '+str(len(moth_tracks))+")",markerfacecolor='k', markersize=10),
                   Line2D([0], [0], linestyle='-', color=getPlotColor(11513300103), label='Vespidae (T = '+str(len(wasp_tracks))+")",markerfacecolor='k', markersize=10)]
                #    Line2D([0], [0], marker="o", color=getPlotColor(11513300101), label='Flowers (T = '+str(flower_count)+")",markerfacecolor='olive', markersize=10, alpha = 0.2)]

# Create the figure
plt.legend(handles=legend_elements, loc=8, ncol=5, frameon=True, fontsize=20, bbox_to_anchor=(0.5, -0.2))
plt.xlabel('X (mm)', fontsize=20)
plt.ylabel('Y (mm)', fontsize=20)
plt.xticks([0, 150, 300, 450, 600],fontsize=20)
plt.yticks([0, 150, 300], fontsize=20)
plt.grid(True, alpha = 0.5)
plt.savefig('/Users/mrat0010/Documents/GitHub/Polytrack_WIP/plots/cv4animals_tracks.png', dpi=300, bbox_inches='tight')



plt.show()

In [47]:
insect_tracks

nframe_11513300100  x0_11513300100  y0_11513300100  area_11513300100  \
0                3521.0          1503.0           110.0             686.0   
1                3522.0          1424.0           145.0             880.0   
2                3523.0          1361.0           153.0             788.0   
3                3524.0          1315.0           154.0            2382.0   
4                3525.0          1276.0           150.0            2501.0   
..                  ...             ...             ...               ...   
540                 NaN             NaN             NaN               NaN   
541                 NaN             NaN             NaN               NaN   
542                 NaN             NaN             NaN               NaN   
543                 NaN             NaN             NaN               NaN   
544                 NaN             NaN             NaN               NaN   

     yadj_11513300100  flower_11513300100  visit_num_11513300100  \
0               970.0                 NaN                    NaN   
1               935.0                 NaN                    NaN   
2               927.0                 NaN                    NaN   
3               926.0                 NaN                    NaN   
4               930.0                 NaN                    NaN   
..                ...                 ...                    ...   
540               NaN                 NaN                    NaN   
541               NaN                 NaN                    NaN   
542               NaN                 NaN                    NaN   
543               NaN                 NaN                    NaN   
544               NaN                 NaN                    NaN   

     sx_11513300100  sy_11513300100  nframe_11513303900  ...  sy_20811271203  \
0       1503.000000      110.000000              4653.0  ...      633.000000   
1       1463.500000      127.500000              4654.0  ...      640.000000   
2       1429.333333      136.000000              4655.0  ...      645.333333   
3       1366.666667      150.666667              4656.0  ...      655.333333   
4       1317.333333      152.333333              4657.0  ...      662.666667   
..              ...             ...                 ...  ...             ...   
540             NaN             NaN                 NaN  ...             NaN   
541             NaN             NaN                 NaN  ...             NaN   
542             NaN             NaN                 NaN  ...             NaN   
543             NaN             NaN                 NaN  ...             NaN   
544             NaN             NaN                 NaN  ...             NaN   

     nframe_71014275703  x0_71014275703  y0_71014275703  area_71014275703  \
0                6961.0          1496.0           875.0             758.0   
1                6962.0          1472.0           807.0            1246.0   
2                6963.0          1460.0           714.0             882.0   
3                6964.0          1467.0           612.0             826.0   
4                6965.0          1483.0           495.0            1107.0   
..                  ...             ...             ...               ...   
540                 NaN             NaN             NaN               NaN   
541                 NaN             NaN             NaN               NaN   
542                 NaN             NaN             NaN               NaN   
543                 NaN             NaN             NaN               NaN   
544                 NaN             NaN             NaN               NaN   

     yadj_71014275703  flower_71014275703  visit_num_71014275703  \
0               875.0                 NaN                    NaN   
1               807.0                 NaN                    NaN   
2               714.0                 NaN                    NaN   
3               612.0                 NaN                    NaN   
4               495.0                

In [50]:
honeybee_tracks = []
hoverfly_tracks = []
moth_tracks = []
wasp_tracks = []

for track in track_list:
    last_digit = int(str(int(track))[-1])
    if last_digit == 0:
        honeybee_tracks.append(track)
    elif last_digit == 1:
        hoverfly_tracks.append(track)
    elif last_digit == 2:
        moth_tracks.append(track)
    elif last_digit == 3:
        wasp_tracks.append(track)

print("Honeybee Tracks:", honeybee_tracks)
print("Hoverfly Tracks:", hoverfly_tracks)
print("Moth Tracks:", moth_tracks)
print("Wasp Tracks:", wasp_tracks)


Honeybee Tracks: [11513300100.0, 11513303900.0, 11513305900.0, 11513313000.0, 11513365400.0, 11712303400.0, 11712394400.0, 31212391200.0, 31212402700.0, 31212413000.0, 31212414100.0, 31714304100.0, 31714310600.0, 31714311800.0, 31714331200.0, 40913341500.0, 40913345500.0, 40913351800.0, 71014302100.0, 71014304300.0, 71014315300.0, 71014323000.0, 81013473200.0]
Hoverfly Tracks: [40913315501.0, 41711574301.0, 41711575901.0, 41712110001.0, 41712111901.0, 41712164101.0]
Moth Tracks: [11513335102.0, 11513335402.0, 11513335502.0, 31212370402.0]
Wasp Tracks: [11513375903.0, 20811265803.0, 20811271003.0, 20811271103.0, 20811271203.0, 71014275703.0]


In [52]:
len(honeybee_tracks)

23

In [56]:
position_list

[115, 117, 208, 312, 317, 409, 417, 710, 810]

In [43]:

insect_tracks = []
first_insect = [101, 'cat', 0, [[10,20,10,]]]
insect_tracks.append(first_insect) 
second_insect = [102, 'dog', 0, [[10,20,10,]]]
insect_tracks.append(second_insect)

insect_num = 102
position = next((i for i, insect in enumerate(insect_tracks) if insect[0] == insect_num), None)



# Add another record for insect 101
insect_tracks[position][3].append([10,82,8,1])
# print(insect_tracks)


insect_tracks[position][3][1].append(4)
print(insect_tracks)



# insect_tracks[insect_position][detections][frame_number].append(4)

[[101, 'cat', 0, [[10, 20, 10]]], [102, 'dog', 0, [[10, 20, 10], [10, 82, 8, 1, 4]]]]


In [44]:
insect = insect_tracks[1][3]
print(insect)

[[10, 20, 10], [10, 82, 8, 1, 4]]


In [31]:
insect_num = 102
position = next((i for i, insect in enumerate(insect_tracks) if insect[0] == insect_num), None)
print("Position of insect_num 101 in insect_tracks:", position)


Position of insect_num 101 in insect_tracks: 1


In [52]:
print(len(insect_tracks[1][3]))

2


In [41]:
a =[0,3,None]
print(a)

[0, 3, None]


In [51]:
a = [[None, None, None, None], [10, 20, 10, None], [10, 82, 8, 1, None], [20, None, None, None],  [30, None, None, None]]

def find_last_valid_position(lst):
    """
    Function to find the index of the last nested list where the 2nd and 3rd values are not None.
    Returns the index of the last valid nested list, or None if no such nested list is found.
    """
    for i in range(len(lst) - 1, -1, -1):  # Iterate backwards over the list
        nested_list = lst[i]
        if len(nested_list) >= 3 and nested_list[1] is not None and nested_list[2] is not None:
            return i  # Return the index of the last valid nested list

# Find the last valid position in list 'a'
last_valid_position = find_last_valid_position(a)

print("Last valid position in list 'a':", last_valid_position)

print(a[last_valid_position][0])

Last valid position in list 'a': 2
10


In [53]:
track = [[3521, 1504, 110, None], [3522, 1419, 141, None], [3523, 1361, 154, None], [3524, 1336, 155, None], [3525, 1297, 148, None], [3526, 1268, 146, None], [3527, 1252, 144, None], [3528, 1248, 148, None], [3529, 1241, 159, None], [3530, 1239, 167, None], [3531, 1243, 162, None], [3532, 1232, 167, None], [3533, 1235, 173, None], [3534, 1227, 172, None], [3535, 1234, 181, None], [3536, 1227, 175, None], [3537, 1241, 180, None], [3538, 1220, 178, None], [3539, 1220, 177, None], [3541, 1219, 176, None], [3542, 1219, 176, None], [3544, 1221, 179, None], [3546, 1221, 178, None], [3549, 1220, 175, None], [3552, 1219, 176, None], [3555, 1229, 163, None], [3557, 1230, 166, None], [3558, 1218, 178, None], [3559, 1215, 178, None], [3562, 1216, 181, None], [3574, 1215, 184, None], [3587, 1226, 178, None], [3589, 1215, 190, None], [3590, 1228, 182, None], [3596, 1224, 180, None], [3597, 1216, 189, None], [3599, 1216, 193, None], [3600, 1214, 191, None], [3604, 1213, 199, None], [3605, 1214, 197, None], [3606, 1213, 198, None], [3626, 1207, 201, None], [3627, 1207, 201, None], [3636, 1207, 198, None]]

In [54]:
last_valid_position = find_last_valid_position(track)

In [56]:
print(last_valid_position)

43


In [57]:
len(track)

44

In [72]:
def find_last_detected_frame(insect_detections: list) -> int:
        
    for i in range(len(insect_detections) - 1, -1, -1):  # Iterate backwards over the list
        nested_list = insect_detections[i]
        if len(nested_list) >= 3 and nested_list[1] is not None and nested_list[2] is not None:
            return i  # Return the index of the last valid nested list

    # last_detected_frame_details = insect_detections[(last_detected_record)]

    # return last_detected_frame_details
    

In [73]:
print(track[find_last_detected_frame(track)])

[3636, 1207, 198, None]


In [1]:
import numpy as np

a = [[  1.001e+09   ,       24       ,  428]]
a = np.array(a)

print(a)


[[1.001e+09 2.400e+01 4.280e+02]]


In [11]:
b = np.delete(a, [0], axis=0)

In [12]:
print(b)

[]


In [ ]:
a = [1,2,3,4,5,6,7,8,9]
b = 5

c =[self.video_frame_height - record[2] for record in insect_track]